In [1]:
!pip install hmmlearn

In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn import hmm
from hmmlearn.hmm import MultinomialHMM
from hmmlearn.hmm import GaussianHMM
from scipy import stats

# Parameters

In [371]:
class Player:
    
    def __init__(self):
        gen_model = hmm.MultinomialHMM(n_components=2, random_state=99)
        gen_model.startprob_ = np.array([0.5, 0.5])
        gen_model.emissionprob_ = np.array([[0.8,0.2],[0.2,0.8]])
        self.model = gen_model
        self. strategy = {
          'always_defect': np.array([[1,0],[1,0]]),
          'always_cooperate' : np.array([[0,1],[0,1]]),
          'average' : np.array([[0.85,0.15],[0.15,0.85]]),
          'stubborn' : np.array([[0.95,0.05],[0.05,0.95]]),
          'ambivalent' : np.array([[0.65,0.35],[0.35,0.65]])
        }
    
    # Instance method
    def expect(self, samples, personality,verbose = False):
      self.model.transmat_ = self.strategy[personality]
      rolls, gen_states = self.model.sample(samples)
      actions = rolls.ravel()
      accuracy = np.mean(actions == gen_states)
      if verbose:
        print(f"{personality} player")
        print(self.model.transmat_)
        print('Hidden State:', gen_states)
        print("Result      :", actions)
        print(f"Opponnent Model Accuracy = {accuracy}")
      return accuracy

    def history(self, row, personality):
      self.model.transmat_ = self.strategy[personality]
      rolls, gen_states = self.model.sample(row)
      actions = rolls.ravel()
      accuracy = np.mean(actions == gen_states)
      return accuracy

    def generate(self, personality, samples):
      self.model.transmat_ = self.strategy[personality]
      rolls, gen_states = self.model.sample(samples)
      actions = rolls.ravel()
      return actions.reshape(-1,1)

    def generate_samples(self, rand, samples, personality):
      self.model.transmat_ = self.strategy[personality]
      rolls, gen_states = self.model.sample(samples, random_state = rand)
      actions = rolls.ravel()
      return actions.reshape(-1,1)
    
    def cooperation_amount(self, actions):
      return np.mean(actions == 1)

def stat_sig(samples, personality):
  trial_df = pd.DataFrame(columns = ['average','stubborn','ambivalent','random_state','data'])
  trial_df['random_state'] = [x for x in range(3,63,3)]
  trial_df['data'] = trial_df['random_state'].apply(P_stat.generate_samples, samples = samples, personality = personality)
  for strategy in ['average', 'stubborn', 'ambivalent']:
    P_stat.model.transmat_ = P_stat.strategy[strategy]
    trial_df[strategy] = -1 / trial_df['data'].apply(P_stat.model.score)
  sum = trial_df[['average', 	'stubborn', 	'ambivalent']].sum(axis=1)
  for k in ['average', 	'stubborn', 	'ambivalent']:
    trial_df[k] = trial_df[k] / sum
  a = b = c = 0.05
  if personality == 'average':
    a = 0.05
    b = c = 0.95
  if personality == 'stubborn':
    b = 0.05
    a = c = 0.95
  if personality == 'ambivalent':
    c = 0.05
    a = b = 0.95
  average_prob = stats.mstats.mquantiles(trial_df['average'], [a])
  stubborn_prob = stats.mstats.mquantiles(trial_df['stubborn'], [b])
  ambivalent_prob = stats.mstats.mquantiles(trial_df['ambivalent'], [c])
  return pd.Series([average_prob, stubborn_prob, ambivalent_prob])

In [311]:
P = Player()

# How we expect each strategy to behave

In [312]:
P.strategy.keys()

dict_keys(['always_defect', 'always_cooperate', 'average', 'stubborn', 'ambivalent'])

In [313]:
for personality in P.strategy.keys():
  P.expect(30, personality, verbose= True)
  print()

always_defect player
[[1 0]
 [1 0]]
Hidden State: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Result      : [1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0]
Opponnent Model Accuracy = 0.7666666666666667

always_cooperate player
[[0 1]
 [0 1]]
Hidden State: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Result      : [1 0 1 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0]
Opponnent Model Accuracy = 0.7

average player
[[0.85 0.15]
 [0.15 0.85]]
Hidden State: [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1]
Result      : [1 0 1 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1 0]
Opponnent Model Accuracy = 0.7666666666666667

stubborn player
[[0.95 0.05]
 [0.05 0.95]]
Hidden State: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Result      : [1 0 1 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0]
Opponnent Model Accuracy = 0.7

ambivalent player
[[0.65 0.35]
 [0.35 0.65]]
Hidden State: [1 1 1 0 1 1 1 

In [314]:
names = [x for x in P.strategy.keys()]
samples = [x for x in range(10,510,10)]
df = pd.DataFrame(columns = names, index = samples)
df.reset_index(inplace=True)
df.rename(columns = {'index':'samples'},inplace=True)
#df

In [315]:
#df['always_defect'] = df['samples'].apply(P.history, personality = 'always_defect')
#df

In [316]:
for i in P.strategy:
  df[i] = df['samples'].apply(P.history, personality = i)
df

,samples,always_defect,always_cooperate,average,stubborn,ambivalent
0,10,0.900000,0.700000,0.600000,0.700000,0.700000
1,20,0.850000,0.650000,0.750000,0.650000,0.650000
2,30,0.766667,0.700000,0.766667,0.700000,0.666667
3,40,0.800000,0.700000,0.750000,0.700000,0.725000
4,50,0.820000,0.740000,0.780000,0.740000,0.780000
5,60,0.816667,0.733333,0.766667,0.733333,0.783333
6,70,0.828571,0.757143,0.785714,0.757143,0.785714
7,80,0.812500,0.775000,0.775000,0.762500,0.812500
8,90,0.822222,0.766667,0.777778,0.777778,0.800000
9,100,0.820000,0.760000,0.780000,0.780000,0.780000


In [318]:
import plotly.express as px
fig = px.line(df, x="samples", y=names)
fig.update_layout(
        title={
        'text': "Opponent Model Accuracy of Each Strategy",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'middle'},
    xaxis_title="# of samples",
    yaxis_title="OM Accuracy",
    legend_title="Hidden State",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
    )
)
fig.show()

# Rating Probability of Random Moves

In [319]:
names = [x for x in P.strategy.keys()]
samples = [x for x in range(0,110,10)]
score_df = pd.DataFrame(columns = names, index = samples)
score_df.reset_index(inplace=True)
score_df.rename(columns = {'index':'cooperation_percentage'},inplace=True)
score_df

,cooperation_percentage,always_defect,always_cooperate,average,stubborn,ambivalent
0,0,NaN,NaN,NaN,NaN,NaN
1,10,NaN,NaN,NaN,NaN,NaN
2,20,NaN,NaN,NaN,NaN,NaN
3,30,NaN,NaN,NaN,NaN,NaN
4,40,NaN,NaN,NaN,NaN,NaN
5,50,NaN,NaN,NaN,NaN,NaN
6,60,NaN,NaN,NaN,NaN,NaN
7,70,NaN,NaN,NaN,NaN,NaN
8,80,NaN,NaN,NaN,NaN,NaN
9,90,NaN,NaN,NaN,NaN,NaN


In [320]:
def generate_test(row):
  prob = row / 100
  x = np.random.choice([0, 1], size=1000, p=[1-prob, prob])
  x = x.reshape(-1,1)
  return x

In [321]:
score_df['data'] = score_df['cooperation_percentage'].apply(generate_test)
score_df

,cooperation_percentage,always_defect,always_cooperate,average,stubborn,ambivalent,data
0,0,NaN,NaN,NaN,NaN,NaN,"[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
1,10,NaN,NaN,NaN,NaN,NaN,"[[1], [0], [0], [0], [0], [0], [0], [0], [1], ..."
2,20,NaN,NaN,NaN,NaN,NaN,"[[1], [1], [0], [0], [0], [1], [0], [0], [0], ..."
3,30,NaN,NaN,NaN,NaN,NaN,"[[0], [0], [0], [0], [0], [1], [0], [0], [1], ..."
4,40,NaN,NaN,NaN,NaN,NaN,"[[0], [0], [0], [0], [0], [1], [0], [1], [0], ..."
5,50,NaN,NaN,NaN,NaN,NaN,"[[0], [1], [1], [1], [0], [0], [0], [1], [0], ..."
6,60,NaN,NaN,NaN,NaN,NaN,"[[1], [0], [0], [1], [0], [0], [1], [0], [1], ..."
7,70,NaN,NaN,NaN,NaN,NaN,"[[1], [1], [1], [0], [0], [1], [1], [0], [0], ..."
8,80,NaN,NaN,NaN,NaN,NaN,"[[1], [1], [1], [0], [1], [1], [1], [1], [0], ..."
9,90,NaN,NaN,NaN,NaN,NaN,"[[1], [0], [1], [1], [1], [1], [1], [1], [1], ..."


In [322]:
for k,v in P.strategy.items():
  P.model.transmat_ = v
  score_df[k] = 1 / (score_df['data'].apply(P.model.score))* -1
score_df

,cooperation_percentage,always_defect,always_cooperate,average,stubborn,ambivalent,data
0,0,0.004472,0.000622,0.002660,0.003648,0.001751,"[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
1,10,0.002679,0.000685,0.002007,0.002375,0.001613,"[[1], [0], [0], [0], [0], [0], [0], [0], [1], ..."
2,20,0.002002,0.000750,0.001693,0.001847,0.001534,"[[1], [1], [0], [0], [0], [1], [0], [0], [0], ..."
3,30,0.001557,0.000840,0.001495,0.001522,0.001466,"[[0], [0], [0], [0], [0], [1], [0], [0], [1], ..."
4,40,0.001247,0.000970,0.001413,0.001374,0.001440,"[[0], [0], [0], [0], [0], [1], [0], [1], [0], ..."
5,50,0.001150,0.001039,0.001376,0.001312,0.001431,"[[0], [1], [1], [1], [0], [0], [0], [1], [0], ..."
6,60,0.000943,0.001297,0.001426,0.001390,0.001450,"[[1], [0], [0], [1], [0], [0], [1], [0], [1], ..."
7,70,0.000888,0.001416,0.001456,0.001447,0.001460,"[[1], [1], [1], [0], [0], [1], [1], [0], [0], ..."
8,80,0.000737,0.002101,0.001745,0.001933,0.001545,"[[1], [1], [1], [0], [1], [1], [1], [1], [0], ..."
9,90,0.000679,0.002782,0.002054,0.002459,0.001626,"[[1], [0], [1], [1], [1], [1], [1], [1], [1], ..."


In [323]:
sum = score_df[['always_defect', 	'always_cooperate', 	'average', 	'stubborn', 	'ambivalent']].sum(axis=1)
sum

0     0.013153
1     0.009359
2     0.007826
3     0.006880
4     0.006445
5     0.006307
6     0.006505
7     0.006667
8     0.008062
9     0.009600
10    0.013153
dtype: float64

In [324]:
for k,v in P.strategy.items():
  score_df[k] = score_df[k] / sum
score_df

,cooperation_percentage,always_defect,always_cooperate,average,stubborn,ambivalent,data
0,0,0.339993,0.047265,0.202245,0.277345,0.133151,"[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
1,10,0.286197,0.073243,0.214458,0.253811,0.172291,"[[1], [0], [0], [0], [0], [0], [0], [0], [1], ..."
2,20,0.255810,0.095879,0.216335,0.235984,0.195991,"[[1], [1], [0], [0], [0], [1], [0], [0], [0], ..."
3,30,0.226316,0.122163,0.217254,0.221234,0.213034,"[[0], [0], [0], [0], [0], [1], [0], [0], [1], ..."
4,40,0.193539,0.150576,0.219243,0.213148,0.223494,"[[0], [0], [0], [0], [0], [1], [0], [1], [0], ..."
5,50,0.182322,0.164728,0.218096,0.207963,0.226892,"[[0], [1], [1], [1], [0], [0], [0], [1], [0], ..."
6,60,0.144899,0.199337,0.219213,0.213657,0.222894,"[[1], [0], [0], [1], [0], [0], [1], [0], [1], ..."
7,70,0.133202,0.212447,0.218418,0.216987,0.218947,"[[1], [1], [1], [0], [0], [1], [1], [0], [0], ..."
8,80,0.091462,0.260638,0.216464,0.239806,0.191631,"[[1], [1], [1], [0], [1], [1], [1], [1], [0], ..."
9,90,0.070733,0.289776,0.213995,0.256124,0.169372,"[[1], [0], [1], [1], [1], [1], [1], [1], [1], ..."


In [326]:
fig = px.line(score_df, x="cooperation_percentage", y=names)
fig.update_layout(
        title={
        'text': "Opponent Model Probability of Random Test Data",
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'middle'},
    yaxis_title="Probability",
    legend_title="Hidden State Strategy",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
    )
)
fig.show()

# Rating Probability of Unknown Generated Move

## 100 moves

In [328]:
names = [x for x in P.strategy.keys()]
score_df = pd.DataFrame(columns = names)
score_df.insert(0,"True Class", names)
score_df['samples'] = 100
score_df

,True Class,always_defect,always_cooperate,average,stubborn,ambivalent,samples
0,always_defect,NaN,NaN,NaN,NaN,NaN,100
1,always_cooperate,NaN,NaN,NaN,NaN,NaN,100
2,average,NaN,NaN,NaN,NaN,NaN,100
3,stubborn,NaN,NaN,NaN,NaN,NaN,100
4,ambivalent,NaN,NaN,NaN,NaN,NaN,100


In [329]:
score_df['data'] = score_df['True Class'].apply(P.generate, samples = 100)
score_df['coopertion_percentage'] = score_df['data'].apply(P.cooperation_amount)
score_df

,True Class,always_defect,always_cooperate,average,stubborn,ambivalent,samples,data,coopertion_percentage
0,always_defect,NaN,NaN,NaN,NaN,NaN,100,"[[1], [0], [0], [0], [0], [0], [0], [0], [0], ...",0.19
1,always_cooperate,NaN,NaN,NaN,NaN,NaN,100,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.76
2,average,NaN,NaN,NaN,NaN,NaN,100,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.52
3,stubborn,NaN,NaN,NaN,NaN,NaN,100,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.62
4,ambivalent,NaN,NaN,NaN,NaN,NaN,100,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.51


In [330]:
for k,v in P.strategy.items():
  P.model.transmat_ = v
  score_df[k] = -1 / score_df['data'].apply(P.model.score)
score_df


,True Class,always_defect,always_cooperate,average,stubborn,ambivalent,samples,data,coopertion_percentage
0,always_defect,0.020948,0.007403,0.017034,0.018733,0.015332,100,"[[1], [0], [0], [0], [0], [0], [0], [0], [0], ...",0.19
1,always_cooperate,0.007889,0.017839,0.016161,0.016859,0.015138,100,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.76
2,average,0.010697,0.011195,0.015765,0.015246,0.015126,100,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.52
3,stubborn,0.009315,0.013251,0.016293,0.017039,0.015175,100,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.62
4,ambivalent,0.010858,0.011024,0.013967,0.013305,0.014523,100,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.51


In [331]:
sum = score_df[['always_defect', 	'always_cooperate', 	'average', 	'stubborn', 	'ambivalent']].sum(axis=1)
for k in P.strategy.keys():
  score_df[k] = score_df[k] / sum
score_df

,True Class,always_defect,always_cooperate,average,stubborn,ambivalent,samples,data,coopertion_percentage
0,always_defect,0.263662,0.093183,0.214399,0.235785,0.192972,100,"[[1], [0], [0], [0], [0], [0], [0], [0], [0], ...",0.19
1,always_cooperate,0.106773,0.241442,0.218724,0.228178,0.204884,100,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.76
2,average,0.157240,0.164561,0.231747,0.224106,0.222346,100,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.52
3,stubborn,0.131069,0.186448,0.229239,0.239739,0.213506,100,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.62
4,ambivalent,0.170517,0.173123,0.219339,0.208945,0.228076,100,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.51


In [332]:
# to check all the columns add to one
score_df[['always_defect', 	'always_cooperate', 	'average', 	'stubborn', 	'ambivalent']].sum(axis=1)

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
dtype: float64

In [333]:
fig = px.bar(score_df[['True Class','always_defect', 	'always_cooperate', 	'average', 	'stubborn', 	'ambivalent']], x="True Class", y=names, barmode='group')
fig.update_layout(
        title={
        'text': "Probability of Detecting Generated Test Data with 100 samples",
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'middle'},
    yaxis_title="Probability",
    legend_title="Predicted Class",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
    )
)
fig.show()

## 1000 moves

In [334]:
names = [x for x in P.strategy.keys()]
score_df = pd.DataFrame(columns = names)
score_df.insert(0,"True Class", names)
score_df['samples'] = 1000
score_df['data'] = score_df['True Class'].apply(P.generate, samples = 1000)
score_df['coopertion_percentage'] = score_df['data'].apply(P.cooperation_amount)
score_df

,True Class,always_defect,always_cooperate,average,stubborn,ambivalent,samples,data,coopertion_percentage
0,always_defect,NaN,NaN,NaN,NaN,NaN,1000,"[[1], [0], [0], [0], [0], [0], [0], [0], [0], ...",0.214
1,always_cooperate,NaN,NaN,NaN,NaN,NaN,1000,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.826
2,average,NaN,NaN,NaN,NaN,NaN,1000,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.521
3,stubborn,NaN,NaN,NaN,NaN,NaN,1000,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.504
4,ambivalent,NaN,NaN,NaN,NaN,NaN,1000,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.531


In [335]:
for k,v in P.strategy.items():
  P.model.transmat_ = v
  score_df[k] = -1 / score_df['data'].apply(P.model.score)
score_df


,True Class,always_defect,always_cooperate,average,stubborn,ambivalent,samples,data,coopertion_percentage
0,always_defect,0.001927,0.000761,0.001687,0.001818,0.001529,1000,"[[1], [0], [0], [0], [0], [0], [0], [0], [0], ...",0.214
1,always_cooperate,0.000731,0.002151,0.001796,0.001989,0.001562,1000,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.826
2,average,0.001059,0.001127,0.001554,0.001526,0.001502,1000,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.521
3,stubborn,0.001086,0.001097,0.001655,0.001721,0.001524,1000,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.504
4,ambivalent,0.001043,0.001144,0.001442,0.001377,0.001462,1000,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.531


In [336]:
sum = score_df[['always_defect', 	'always_cooperate', 	'average', 	'stubborn', 	'ambivalent']].sum(axis=1)
for k in P.strategy.keys():
  score_df[k] = score_df[k] / sum
score_df

,True Class,always_defect,always_cooperate,average,stubborn,ambivalent,samples,data,coopertion_percentage
0,always_defect,0.249531,0.098596,0.218437,0.235453,0.197983,1000,"[[1], [0], [0], [0], [0], [0], [0], [0], [0], ...",0.214
1,always_cooperate,0.088865,0.261400,0.218273,0.241685,0.189777,1000,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.826
2,average,0.156450,0.166468,0.229626,0.225489,0.221967,1000,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.521
3,stubborn,0.153298,0.154930,0.233617,0.243007,0.215148,1000,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.504
4,ambivalent,0.161305,0.176916,0.222894,0.212811,0.226075,1000,"[[1], [0], [1], [0], [0], [1], [1], [1], [1], ...",0.531


In [337]:
fig = px.bar(score_df[['True Class','always_defect', 	'always_cooperate', 	'average', 	'stubborn', 	'ambivalent']], x="True Class", y=names, barmode='group')
fig.update_layout(
        title={
        'text': "Probability of Detecting Generated Test Data with 1000 samples",
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'middle'},
    yaxis_title="Probability",
    legend_title="Predicted Class",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
    )
)
fig.show()

## Testing for Statistical Signficance

### Stubborn

In [347]:
P_stat = Player()
objective= 'stubborn'
trial_df = pd.DataFrame(columns = ['average','stubborn','ambivalent','random_state','data']) #intialize dataframe
trial_df['random_state'] = [x for x in range(3,93,3)] #set random states
trial_df['data'] = trial_df['random_state'].apply(P_stat.generate_samples, samples = 100, personality = objective) #generate samples from stubborn strategy
for strategy in ['average', 'stubborn', 'ambivalent']:   #score each strategy
  P_stat.model.transmat_ = P_stat.strategy[strategy]
  trial_df[strategy] = -1 / trial_df['data'].apply(P_stat.model.score)
sum = trial_df[['average', 	'stubborn', 	'ambivalent']].sum(axis=1)
for k in ['average', 	'stubborn', 	'ambivalent']:
  trial_df[k] = trial_df[k] / sum
trial_df

,average,stubborn,ambivalent,random_state,data
0,0.338435,0.348337,0.313228,3,"[[1], [1], [1], [1], [1], [0], [0], [0], [0], ..."
1,0.340282,0.352425,0.307293,6,"[[1], [0], [1], [1], [1], [1], [1], [1], [1], ..."
2,0.336926,0.339172,0.323902,9,"[[0], [0], [0], [0], [0], [1], [0], [1], [1], ..."
3,0.339441,0.342926,0.317633,12,"[[0], [0], [1], [0], [0], [1], [1], [0], [0], ..."
4,0.337582,0.336088,0.326330,15,"[[0], [1], [1], [1], [1], [1], [1], [1], [0], ..."
5,0.337221,0.345966,0.316814,18,"[[1], [0], [1], [1], [0], [1], [0], [1], [1], ..."
6,0.338373,0.366161,0.295466,21,"[[0], [0], [0], [0], [0], [1], [0], [1], [1], ..."
7,0.334167,0.358769,0.307064,24,"[[1], [1], [1], [1], [1], [1], [1], [1], [1], ..."
8,0.336769,0.344042,0.319189,27,"[[1], [1], [1], [0], [0], [1], [1], [0], [0], ..."
9,0.337489,0.341416,0.321095,30,"[[1], [0], [1], [1], [1], [1], [1], [0], [1], ..."


In [348]:
stubborn = trial_df[['average', 	'stubborn', 	'ambivalent']]
stubborn.describe()

,average,stubborn,ambivalent
count,30.000000,30.000000,30.000000
mean,0.338309,0.348270,0.313421
std,0.002407,0.009730,0.009711
min,0.334167,0.333526,0.291474
25%,0.336779,0.340765,0.307521
50%,0.338205,0.346138,0.316636
75%,0.339471,0.354110,0.319859
max,0.342840,0.373973,0.326664


In [349]:
##bootstrapping
##true_class = 30 trials
stubborn_df = pd.DataFrame(columns = ['samples','average','stubborn','ambivalent'])
stubborn_df['samples'] = [x for x in range(100,1100,100)]
stubborn_df[['average','stubborn','ambivalent']] = stubborn_df['samples'].apply(stat_sig, personality = objective)
for i in ['average','stubborn','ambivalent']:
  stubborn_df[i] = stubborn_df[i].apply(np.asscalar)
stubborn_df

,samples,average,stubborn,ambivalent
0,100,0.342022,0.336315,0.326527
1,200,0.340276,0.335563,0.327235
2,300,0.339492,0.335803,0.326374
3,400,0.338943,0.338169,0.323653
4,500,0.339604,0.338766,0.322670
5,600,0.339914,0.339917,0.322115
6,700,0.339973,0.339633,0.323047
7,800,0.339784,0.339978,0.322366
8,900,0.339531,0.340700,0.321258
9,1000,0.339299,0.340772,0.321185


In [350]:
names = ['average','stubborn','ambivalent']
fig = px.line(stubborn_df,  x="samples", y= names)
fig.update_layout(
        title={
        'text': "Stubborn Strategy: Size of Samples to Achieve 95% Statistical Signficance",
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'middle'},
    yaxis_title="Probability",
    xaxis_title = "# of Samples from Stubborn Strategy",
    legend_title="Credible Interval Bounds",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
    )
)
fig.add_annotation(x=900, y= 	0.340700 	, text="Lower-bounds of stubborn > upper-bounds", showarrow=True, arrowhead=1)
fig.show()

### Ambivalent

In [351]:
P_stat = Player()
objective= 'ambivalent'
trial_df = pd.DataFrame(columns = ['average','stubborn','ambivalent','random_state','data']) #intialize dataframe
trial_df['random_state'] = [x for x in range(3,93,3)] #set random states
trial_df['data'] = trial_df['random_state'].apply(P_stat.generate_samples, samples = 100, personality = objective) #generate samples from stubborn strategy
for strategy in ['average', 'stubborn', 'ambivalent']:   #score each strategy
  P_stat.model.transmat_ = P_stat.strategy[strategy]
  trial_df[strategy] = -1 / trial_df['data'].apply(P_stat.model.score)
sum = trial_df[['average', 	'stubborn', 	'ambivalent']].sum(axis=1)
for k in ['average', 	'stubborn', 	'ambivalent']:
  trial_df[k] = trial_df[k] / sum
trial_df

,average,stubborn,ambivalent,random_state,data
0,0.335399,0.315970,0.348631,3,"[[1], [0], [1], [0], [0], [0], [0], [1], [0], ..."
1,0.336066,0.315573,0.348361,6,"[[1], [0], [0], [0], [0], [0], [0], [1], [1], ..."
2,0.333499,0.334152,0.332348,9,"[[0], [0], [0], [0], [0], [1], [0], [1], [1], ..."
3,0.338050,0.335173,0.326777,12,"[[0], [0], [1], [0], [0], [0], [1], [0], [0], ..."
4,0.334114,0.312213,0.353673,15,"[[0], [0], [0], [0], [0], [1], [1], [1], [0], ..."
5,0.335043,0.324463,0.340494,18,"[[1], [0], [1], [1], [0], [1], [0], [0], [1], ..."
6,0.337774,0.326955,0.335271,21,"[[0], [0], [0], [0], [0], [1], [0], [1], [1], ..."
7,0.335844,0.332018,0.332137,24,"[[1], [1], [1], [1], [0], [0], [1], [1], [1], ..."
8,0.338092,0.324850,0.337058,27,"[[1], [1], [1], [1], [1], [1], [1], [0], [0], ..."
9,0.337488,0.322318,0.340194,30,"[[1], [0], [1], [1], [1], [0], [0], [0], [1], ..."


In [352]:
stubborn = trial_df[['average', 	'stubborn', 	'ambivalent']]
stubborn.describe()

,average,stubborn,ambivalent
count,30.000000,30.000000,30.000000
mean,0.336079,0.321951,0.341970
std,0.002201,0.006856,0.007555
min,0.331113,0.312020,0.324688
25%,0.334132,0.316524,0.337689
50%,0.335955,0.321811,0.342042
75%,0.337738,0.326008,0.348434
max,0.340057,0.335255,0.353673


In [353]:
##bootstrapping
##true_class = 30 trials
stubborn_df = pd.DataFrame(columns = ['samples','average','stubborn','ambivalent'])
stubborn_df['samples'] = [x for x in range(100,1100,100)]
stubborn_df[['average','stubborn','ambivalent']] = stubborn_df['samples'].apply(stat_sig, personality = objective)
for i in ['average','stubborn','ambivalent']:
  stubborn_df[i] = stubborn_df[i].apply(np.asscalar)
stubborn_df

,samples,average,stubborn,ambivalent
0,100,0.338130,0.334754,0.328975
1,200,0.339164,0.330308,0.332320
2,300,0.338691,0.327244,0.335240
3,400,0.337448,0.324837,0.338241
4,500,0.337420,0.326031,0.336661
5,600,0.337391,0.326450,0.336709
6,700,0.337285,0.325010,0.338140
7,800,0.337196,0.323969,0.339708
8,900,0.336950,0.323646,0.340099
9,1000,0.336639,0.323138,0.340661


In [354]:
names = ['average','stubborn','ambivalent']
fig = px.line(stubborn_df,  x="samples", y= names)
fig.update_layout(
        title={
        'text': "Ambivalent Strategy: Size of Samples to Achieve 95% Statistical Signficance",
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'middle'},
    yaxis_title="Probability",
    xaxis_title = "# of Samples from Ambivalent Strategy",
    legend_title="Credible Interval Bounds",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
    )
)
fig.add_annotation(x=700, y= 0.338140 	, text="Lower-bounds of ambivalent > upper-bounds", showarrow=True, arrowhead=1)
fig.show()

### Average

In [372]:
P_stat = Player()
objective= 'average'
trial_df = pd.DataFrame(columns = ['average','stubborn','ambivalent','random_state','data']) #intialize dataframe
trial_df['random_state'] = [x for x in range(3,93,3)] #set random states
trial_df['data'] = trial_df['random_state'].apply(P_stat.generate_samples, samples = 100, personality = objective) #generate samples from stubborn strategy
for strategy in ['average', 'stubborn', 'ambivalent']:   #score each strategy
  P_stat.model.transmat_ = P_stat.strategy[strategy]
  trial_df[strategy] = -1 / trial_df['data'].apply(P_stat.model.score)
sum = trial_df[['average', 	'stubborn', 	'ambivalent']].sum(axis=1)
for k in ['average', 	'stubborn', 	'ambivalent']:
  trial_df[k] = trial_df[k] / sum
trial_df

,average,stubborn,ambivalent,random_state,data
0,0.338970,0.334479,0.326551,3,"[[1], [1], [1], [0], [0], [0], [0], [0], [0], ..."
1,0.338612,0.323000,0.338388,6,"[[1], [0], [0], [0], [0], [0], [0], [1], [0], ..."
2,0.336150,0.325715,0.338134,9,"[[0], [0], [0], [0], [0], [1], [0], [1], [1], ..."
3,0.339371,0.321293,0.339335,12,"[[0], [0], [1], [0], [0], [1], [1], [0], [0], ..."
4,0.339167,0.320483,0.340350,15,"[[0], [0], [0], [0], [0], [1], [1], [1], [0], ..."
5,0.337935,0.330518,0.331547,18,"[[1], [0], [1], [1], [0], [1], [0], [0], [1], ..."
6,0.339394,0.340407,0.320199,21,"[[0], [0], [0], [0], [0], [1], [0], [1], [1], ..."
7,0.340707,0.334501,0.324793,24,"[[1], [1], [1], [1], [0], [0], [1], [0], [0], ..."
8,0.338327,0.323209,0.338464,27,"[[1], [1], [1], [1], [1], [1], [1], [1], [1], ..."
9,0.341265,0.319558,0.339177,30,"[[1], [0], [1], [1], [1], [0], [0], [0], [1], ..."


In [373]:
average = trial_df[['average', 	'stubborn', 	'ambivalent']]
average.describe()

,average,stubborn,ambivalent
count,30.000000,30.000000,30.000000
mean,0.338834,0.333166,0.328000
std,0.002870,0.008936,0.009039
min,0.334067,0.319558,0.302724
25%,0.336394,0.325633,0.323880
50%,0.338634,0.332706,0.328337
75%,0.341076,0.339117,0.335391
max,0.345250,0.356077,0.340350


In [374]:
##bootstrapping
##true_class = 30 trials
average_df = pd.DataFrame(columns = ['samples','average','stubborn','ambivalent'])
average_df['samples'] = [x for x in range(100,1100,100)]
average_df[['average','stubborn','ambivalent']] = average_df['samples'].apply(stat_sig, personality = objective)
for i in ['average','stubborn','ambivalent']:
  average_df[i] = average_df[i].apply(np.asscalar)
average_df

,samples,average,stubborn,ambivalent
0,100,0.335113,0.346149,0.339934
1,200,0.336356,0.342657,0.341226
2,300,0.337003,0.346496,0.336147
3,400,0.336904,0.344170,0.337248
4,500,0.337656,0.343562,0.335566
5,600,0.337891,0.341306,0.336095
6,700,0.337858,0.339124,0.335575
7,800,0.337901,0.337608,0.336099
8,900,0.337961,0.336846,0.336580
9,1000,0.337998,0.336828,0.335653


In [376]:
names = ['average','stubborn','ambivalent']
fig = px.line(average_df,  x="samples", y= names)
fig.update_layout(
        title={
        'text': "Average Strategy: Size of Samples to Achieve 95% Statistical Signficance",
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'middle'},
    yaxis_title="Probability",
    xaxis_title = "# of Samples from Average Strategy",
    legend_title="Credible Interval Bounds",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
    )
)
fig.add_annotation(x=800, y= 0.338140 	, text="Lower-bounds of average > upper-bounds", showarrow=True, arrowhead=1)
fig.show()